In [1]:
import os
from dotenv import load_dotenv
import base64
import re
import json

import openai
from openai import OpenAI

In [4]:
!ls ../images

Aカプラー.png                         Sカプラー.png
Dカプラー.png                         UKパイント.png
FROW_CONTROL_FAUCET.png               USパイント.png
Gカプラー.png                         ヴァイツェングラス.png
KEY-KEGカプラー.png                   グラグラ.png
STNDARD_FAUCET.png                    足付き.png
STOUT_FAUCET.png


In [6]:
image_dir = '../images'
img_grass = ['グラグラ.png', '足付き.png', 'USパイント.png', 'UKパイント.png', 'ヴァイツェングラス.png']
img_kap = ['Sカプラー.png', 'Dカプラー.png', 'Gカプラー.png', 'KEY-KEGカプラー.png', 'Aカプラー.png']
img_faucet = ['STNDARD_FAUCET.png', 'FROW_CONTROL_FAUCET.png', 'STOUT_FAUCET.png']

In [7]:
load_dotenv('.env')

client = OpenAI(api_key=os.getenv('openai-key-20241027'))
client

In [13]:
# Test

img = image_dir + '/' + 'グラグラ.png'

vision_model = "gpt-4o-mini"

prompt = '''
グラス画像が入力されます。写っているグラスの形状を80トークン程度で詳しく述べてください。
'''

# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


print('>>>>> ', img)
    
# Path to your image
image_path = os.path.join(image_dir, img)
    
# Getting the base64 string
base64_image = encode_image(image_path)
#print(type(base64_image), len(base64_image))

response = client.chat.completions.create(
    model=vision_model,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt,
                },
                {
                    "type": "image_url",
                    "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"},
                },
            ],
        }
    ],
)
print(response.choices[0].message.content)

>>>>>  ../images/グラグラ.png
このグラスは、スリムなデザインで、高さがあり、底が広く、上に向かって少し狭まる形状をしています。透明なガラス素材で作られ、内側にビールやドリンクが注がれることを想定しています。正面には「CRAFT BEER MARKET」のロゴが描かれた青いラベルが貼られています。シンプルでありながらスタイリッシュな印象を与えるグラスです。


### グラス

In [14]:
%%time

responses_grass = {}

vision_model = "gpt-4o-mini"

prompt = '''
グラス画像が入力されます。写っているグラスの形状を80トークン程度で詳しく述べてください。
'''

# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

for img in img_grass:
    img = image_dir + '/' + img
    print('>>>>> ', img)
    
    # Path to your image
    image_path = os.path.join(image_dir, img)
    
    # Getting the base64 string
    base64_image = encode_image(image_path)
    #print(type(base64_image), len(base64_image))

    response = client.chat.completions.create(
        model=vision_model,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt,
                    },
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"},
                    },
                ],
            }
        ],
    )
    responses_grass[img] = response.choices[0].message.content
    #print(responses[img])

>>>>>  ../images/グラグラ.png
>>>>>  ../images/足付き.png
>>>>>  ../images/USパイント.png
>>>>>  ../images/UKパイント.png
>>>>>  ../images/ヴァイツェングラス.png
CPU times: user 67.1 ms, sys: 16.5 ms, total: 83.6 ms
Wall time: 36.2 s


In [15]:
responses_grass

{'../images/グラグラ.png': 'このグラスは、細長くスリムなデザインで、透明なガラス素材でできています。上部は広がっておらず、ストレートな形状が特徴です。底部はしっかりした円形で安定感があります。ガラスの中ほどにはロゴが印刷されており、全体的にシンプルでありながら上品な印象を与えます。特にビールを注ぐために適しているように見えます。',
 '../images/足付き.png': 'このグラスは、足付きのデザインが特徴的で、上部が広がったシルエットを持っています。グラスの底はしっかりとした円形の台座に固定されており、高さがありながらも安定感があります。グラスの側面は滑らかで透明感があり、飲み物の色を美しく引き立てます。全体的に洗練された印象を与え、特にビールやカクテルに適しています。',
 '../images/USパイント.png': '画像に写っているグラスは、透明なガラス製のUSパイントグラスです。形状は円筒形で直線的なデザインが特徴で、上部が広がっているものの、全体的にスリムな印象を与えます。底部はしっかりとした安定感があり、平らな厚い底が施されています。グラスの側面には「CRAFT BEER MARKET」のロゴが印刷されており、カジュアルな雰囲気を演出しています。',
 '../images/UKパイント.png': 'このグラスは、UKスタイルのパイントグラスで、主にビールやサイダーを提供するために使用されます。特徴的な形状は、底部がしっかりとした平らな部分から始まり、上部に向かって徐々に広がり、ふっくらとした輪郭を持っています。透明なガラス素材により、中身がよく見え、飲み物の色や泡立ちを楽しむことができます。デザインはシンプルで、カジュアルな雰囲気を演出し、親しみやすい印象を与えます。',
 '../images/ヴァイツェングラス.png': 'このグラスは、ヴァイツェングラスと呼ばれる特有の形状をしています。上部は広がり、口が大きく開いているのが特徴です。中間部分は細く絞られており、底はしっかりとした台座になっています。透明なガラス素材で、内側に施された細かな凹凸が飲み物の泡立ちを促進します。全体的に優雅で洗練されたデザインで、主にビールを楽しむために使用されることが多いです。'}

### カプラー

In [19]:
%%time

responses_kap = {}

vision_model = "gpt-4o-mini"

prompt = '''
カプラー（ケグとビアラインを繋ぐ器具）の画像が入力されます。写っているカプラーの形状を文章等参考にしながら100トークン程度で詳しく述べてください。
'''

# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

for img in img_kap:
    img = image_dir + '/' + img
    print('>>>>> ', img)
    
    # Path to your image
    image_path = os.path.join(image_dir, img)
    
    # Getting the base64 string
    base64_image = encode_image(image_path)
    #print(type(base64_image), len(base64_image))

    response = client.chat.completions.create(
        model=vision_model,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt,
                    },
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"},
                    },
                ],
            }
        ],
    )
    responses_kap[img] = response.choices[0].message.content

>>>>>  ../images/STNDARD_FAUCET.png
>>>>>  ../images/FROW_CONTROL_FAUCET.png
>>>>>  ../images/STOUT_FAUCET.png
CPU times: user 31.4 ms, sys: 6.17 ms, total: 37.5 ms
Wall time: 11.8 s


In [18]:
responses_kap

{'../images/Sカプラー.png': 'この画像に写っているカプラーは「Sカプラー」と呼ばれ、主にケグとビアラインを接続するための器具です。形状は、上部にハンドルがあり、操作しやすいデザインが特徴です。また、口径の特定が重要で、ハンドルでの判断ではなく、口径を基に見分けることが強調されています。下部には接続部分があり、一般的に使用されるアサヒ、サッポロ、サントリー系に対応しています。全体的に金属製で、耐久性が高いです。',
 '../images/Dカプラー.png': 'この画像に映っているDカプラーは、主に海外ビールに使用される接続器具です。カプラーは、金属とプラスチックの組み合わせで構成されており、上部にハンドルがあり、操作によって接続や切断が可能です。下部には円形の接続部があり、ビアラインとケグを確実に接続します。口径が確認できるデザインで、適切なサイズを選定することが強調されています。全体として、耐久性のある材料で製造されており、ビールの流れをスムーズに保つ役割を果たします。',
 '../images/Gカプラー.png': 'Gカプラーは、ビールの樽（ケグ）とビアラインを接続するための重要な器具です。このカプラーは、金属製の体に、栓を開けるためのレバーが付いており、操作が簡単です。カプラーの先端には、接続用のホースが結合できる口があり、その周囲にはしっかりとしたシールが施されています。カプラー内部には、液体が流れるための通路があり、飲料の供給がスムーズに行われる構造になっています。使用例としては、キリン系やヤッホーブルーイング、大山Gビールなどで一般的に用いられています。',
 '../images/KEY-KEGカプラー.png': 'このカプラーは、主に海外ビール用に設計された「KEY-KEG」タイプです。全体的に金属製の構造を持ち、耐久性と信頼性があります。上部には操作ハンドルがあり、簡単に接続や解除が可能です。下部には接続部があり、ホースやビアラインとしっかりと結合されます。内部には特殊なディスクがあり、ビールの流れを円滑に保つ役割を果たしています。口径の設計も考慮されており、特にウイスキーや他の飲料に適しています。このカプラーは、機能性とユーザビリティを両立させた設計となっています。',
 '../images/Aカプラー

###　FAUCET

In [22]:
%%time

responses_faucet = {}

vision_model = "gpt-4o-mini"

prompt = '''
Faucet/フォーセット（ビールを注ぐ蛇口）の画像が入力されます。写っているFaucetの形状を50トークン程度で詳しく述べてください。
'''

# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

for img in img_faucet:
    img = image_dir + '/' + img
    print('>>>>> ', img)
    
    # Path to your image
    image_path = os.path.join(image_dir, img)
    
    # Getting the base64 string
    base64_image = encode_image(image_path)
    #print(type(base64_image), len(base64_image))

    response = client.chat.completions.create(
        model=vision_model,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt,
                    },
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"},
                    },
                ],
            }
        ],
    )
    responses_faucet[img] = response.choices[0].message.content

>>>>>  ../images/STNDARD_FAUCET.png
>>>>>  ../images/FROW_CONTROL_FAUCET.png
>>>>>  ../images/STOUT_FAUCET.png
CPU times: user 28.2 ms, sys: 6.28 ms, total: 34.5 ms
Wall time: 11.4 s


In [23]:
responses_faucet

{'../images/STNDARD_FAUCET.png': 'このフォーセットは、金属製の光沢あるデザインが特徴で、上部には調整用のハンドルがあります。底部は蛇口の先端に向かってカーブしており、泡立たないようにビールを注ぎやすい構造になっています。取り付け部分はスレッド形状で、他の機器に接続可能です。全体的にシンプルで機能的な形状です。',
 '../images/FROW_CONTROL_FAUCET.png': 'このフォーセットは、金属製で光沢のある仕上げが特徴です。上部にはネジがあり、取り付け部分が見えます。横には流量調整のレバーがあり、必要に応じてビールの流れを細かく調整できます。全体的に直線的なデザインで、機能的かつシンプルな形状です。',
 '../images/STOUT_FAUCET.png': 'このフォーセットは、スタウト用のビール蛇口で、金属製の光沢のあるデザインが特徴です。上部には注ぎ口があり、細長くスリムな形状をしています。また、蛇口の下部にはホース接続のための口があり、操作するためのレバーも備わっています。全体的にモダンな印象を与えるエレガントなフォルムです。'}